In [1]:
import torchvision

In [2]:
alexnet=torchvision.models.alexnet()

In [3]:
import torch.nn as nn

In [4]:
#input size 1*28*28, 10 classes
class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5, self).__init()
        
        self.features=nn.Sequential(
            nn.Conv2d(1, 6, kernel_size=5, stride=1),
            nn.ReLU(True),
            nn.MaxPool2d(size=2, stride=2, dilation=1),
            nn.Conv2d(6, 16, kernel_size=5, stride=1),
            nn.ReLU(True),
            nn.MaxPool2d(size=2, stride=2, dilation=1),)
        
        self.classifier=nn.Sequential(
            nn.Linear(256, 120),
            nn.ReLU(True),
            nn.Linear(120, 84),
            nn.ReLU(True),
            nn.Linear(84, 10))
        
        
    def forward(self, x):
        x=self.features(x)
        x=x.view(x.size(0),-1)
        x=self.classifier(x)
        

In [6]:
#input size 3*244*244, 1000 classes
class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet, self).__init__()
    
        self.features=nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, podding=2),
            nn.ReLU(True),
            nn.MaxPool2d(size=3, stride=2, dilation=1),
            nn.Conv2d(64, 192, kernel_size=5, stride=1, padding=2),
            nn.ReLU(True),
            nn.MaxPool2d(size=3, stride=2, dilation=1),
            nn.Conv2d(192, 384, kernel_size=3, stride=1, padding=1),
            nn.ReLU(True),
            nn.Conv2d(384, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(True),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(True),
            nn.MaxPool2d(size=3, stride=2, dilation=1))
    
        self.classifier=nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(9216, 4096),
            nn.ReLU(True),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Linear(4096, 1000))
    
    def forward(self, x):
        x=self.features(x)
        x=x.view(x.size(0), -1)
        x=self.classifier(x)
    


In [7]:
import PIL, torch

In [14]:
#load and normalize the image
to_tensor=torchvision.transforms.Compose([torchvision.transforms.Resize((244, 244)), torchvision.transforms.ToTensor()])
img=to_tensor(PIL.Image.open('C:/Users/Administrator/Desktop/dog.jpg'))

In [25]:
img=img.unsqueeze(0)
img=0.5+0.5*(img-img.mean())/img.std()

In [26]:
img.shape

torch.Size([1, 3, 244, 244])

In [27]:
#load and evaluate the network

alexnet=torchvision.models.alexnet(pretrained=True)
alexnet.eval()

output=alexnet(img)

In [28]:
#Prints the classes
scores,indexes=output.view(-1).sort(descending=True)

In [32]:
class_names=eval(open('C:/Users/Administrator/Desktop/imagenet1000_clsidx_to_labels.txt', 'r').read())

In [33]:
for k in range(15):
    print('%.02f' %scores[k].item(), class_names[indexes[k].item()])

13.71 golden retriever
13.46 clumber, clumber spaniel
12.98 English setter
12.13 Great Pyrenees
11.67 cocker spaniel, English cocker spaniel, cocker
11.08 Labrador retriever
11.06 otterhound, otter hound
10.85 Brittany spaniel
10.45 basset, basset hound
10.37 kuvasz
10.28 Sussex spaniel
10.27 Old English sheepdog, bobtail
10.12 Tibetan terrier, chrysanthemum dog
9.71 tennis ball
9.41 Welsh springer spaniel


In [41]:
#transforms a series of layers from a standard convnet to make it fully conv

def convolutionize(layers, input_size):
    
    result_layers=[]
    x=torch.zeros((1, )+input_size)
    
    for m in layers:
        if isinstance(m ,torch.nn.Linear):
            n=torch.nn.Conv2d(in_channels=x.size(1),
                             out_channels=m.weight.size(0),
                             kernel_size=(x.size(2), x.size(3)))
            with torch.no_grad():
                n.weight.view(-1).copy_(n.weight.view(-1))
                n.bias.view(-1).copy_(m.bias.view(-1))
                m=n
            
        result_layers.append(m)
        x=m(x)
                
    return result_layers

In [42]:
#to aaply this to AlexNet:

model=torchvision.models.alexnet(pretrained=True)
print(model)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [43]:
layers=list(model.features)+list(model.classifier)

In [44]:
model=nn.Sequential(*convolutionize(layers, (3, 244, 244)))

In [45]:
print(model)

Sequential(
  (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
  (1): ReLU(inplace=True)
  (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (4): ReLU(inplace=True)
  (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (7): ReLU(inplace=True)
  (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (9): ReLU(inplace=True)
  (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (13): Dropout(p=0.5, inplace=False)
  (14): Conv2d(256, 4096, kernel_size=(6, 6), stride=(1, 1))
  (15): ReLU(inplace=True)
  (16): Dropout(p=0.5, inplace=False)
  (17): Conv2d(4096, 4096, kernel_size=(1, 1), stride=(1, 1))
  (18): Re